In [1]:
!pip install pyLDAvis

In [2]:

import numpy as np

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns

# Model 1: Food and Animals

In [3]:
doc_1 = 'I like to eat broccoli and bananas.'
doc_2 = 'I ate a banana and spinach smoothie for breakfast.'
doc_3 = 'Chinchillas and kittens are cute.'
doc_4 = 'My sister adopted a kitten yesterday.'
doc_5 = 'Look at this cute hamster munching on a piece of broccoli.'

## Step 1: Preprocess our text.

In [4]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # stemming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- we're keeping our words distinct

In [5]:
text_process(doc_1)

['like', 'eat', 'broccoli', 'banana']

In [6]:
texts = [text_process(doc_1),
         text_process(doc_2),
         text_process(doc_3),
         text_process(doc_4),
         text_process(doc_5)]

In [7]:
texts

[['like', 'eat', 'broccoli', 'banana'],
 ['ate', 'banana', 'spinach', 'smoothi', 'breakfast'],
 ['chinchilla', 'kitten', 'cute'],
 ['sister', 'adopt', 'kitten', 'yesterday'],
 ['look', 'cute', 'hamster', 'munch', 'piec', 'broccoli']]

## Step 2: Fit LDA Model.

In [8]:
from gensim import corpora, models
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

np.random.seed(42)

In [9]:
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = models.ldamodel.LdaModel(corpus,                     # pass in our corpus
                                    id2word = dictionary,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 2,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01) # only include topics above this probability threshold

In [10]:
for text in texts:
    print(dictionary.doc2bow(text))

[(0, 1), (1, 1), (2, 1), (3, 1)]
[(0, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(8, 1), (9, 1), (10, 1)]
[(10, 1), (11, 1), (12, 1), (13, 1)]
[(1, 1), (9, 1), (14, 1), (15, 1), (16, 1), (17, 1)]


## Step 3: Visualize LDA model.

In [11]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.064057  0.0       1        1  57.160194
0     -0.064057  0.0       2        1  42.839806, topic_info=     Category      Freq        Term     Total  loglift  logprob
term                                                           
0     Default  1.000000      banana  1.000000  18.0000  18.0000
4     Default  1.000000         ate  1.000000  17.0000  17.0000
5     Default  1.000000   breakfast  1.000000  16.0000  16.0000
7     Default  1.000000     spinach  1.000000  15.0000  15.0000
6     Default  1.000000     smoothi  1.000000  14.0000  14.0000
3     Default  1.000000        like  1.000000  13.0000  13.0000
2     Default  1.000000         eat  1.000000  12.0000  12.0000
9     Default  1.000000        cute  1.000000  11.0000  11.0000
10    Default  1.000000      kitten  1.000000  10.0000  10.0000
16    Default  1.000000       munch  1.000000   9.0000   9.0000
17    Default  1.000000        piec  1.000000   8.0000   8.0000
15    Default  1.000000        look  1.000000   7.0000   7.0000
14    Default  1.000000     hamster  1.000000   6.0000   6.0000
1     Default  1.000000    broccoli  1.000000   5.0000   5.0000
8     Default  1.000000  chinchilla  1.000000   4.0000   4.0000
11    Default  1.000000       adopt  1.000000   3.0000   3.0000
13    Default  1.000000   yesterday  1.000000   2.0000   2.0000
12    Default  1.000000      sister  1.000000   1.0000   1.0000
9      Topic1  1.430452        cute  1.701684   0.3857  -2.1737
10     Topic1  1.401929      kitten  1.698658   0.3673  -2.1939
16     Topic1  0.860746       munch  1.124604   0.2919  -2.6817
17     Topic1  0.860512        piec  1.124579   0.2917  -2.6820
15     Topic1  0.860079        look  1.124533   0.2912  -2.6825
14     Topic1  0.859659     hamster  1.124489   0.2908  -2.6829
8      Topic1  0.852278  chinchilla  1.123706   0.2828  -2.6916
11     Topic1  0.830066       adopt  1.121350   0.2585  -2.7180
13     Topic1  0.827154   yesterday  1.121041   0.2553  -2.7215
12     Topic1  0.824012      sister  1.120708   0.2518  -2.7253
1      Topic1  0.874738    broccoli  1.642745  -0.0709  -2.6656
2      Topic1  0.302878         eat  1.065437  -0.6985  -3.7262
3      Topic1  0.300219        like  1.065155  -0.7071  -3.7350
6      Topic1  0.297720     smoothi  1.064890  -0.7152  -3.7433
7      Topic1  0.296370     spinach  1.064747  -0.7196  -3.7479
5      Topic1  0.296223   breakfast  1.064731  -0.7201  -3.7484
4      Topic1  0.294679         ate  1.064567  -0.7251  -3.7536
0      Topic1  0.305530      banana  1.582375  -1.0853  -3.7174
0      Topic2  1.276845      banana  1.582375   0.6332  -1.9989
4      Topic2  0.769888         ate  1.064567   0.5236  -2.5049
5      Topic2  0.768508   breakfast  1.064731   0.5217  -2.5066
7      Topic2  0.768377     spinach  1.064747   0.5215  -2.5068
6      Topic2  0.767170     smoothi  1.064890   0.5198  -2.5084
3      Topic2  0.764936        like  1.065155   0.5166  -2.5113
2      Topic2  0.762559         eat  1.065437   0.5132  -2.5144
1      Topic2  0.768007    broccoli  1.642745   0.0874  -2.5073
12     Topic2  0.296696      sister  1.120708  -0.4813  -3.4584
13     Topic2  0.293887   yesterday  1.121041  -0.4911  -3.4679
11     Topic2  0.291285       adopt  1.121350  -0.5003  -3.4768
8      Topic2  0.271428  chinchilla  1.123706  -0.5730  -3.5474
14     Topic2  0.264829     hamster  1.124489  -0.5983  -3.5720
15     Topic2  0.264454        look  1.124533  -0.5998  -3.5734
17     Topic2  0.264068        piec  1.124579  -0.6013  -3.5749
16     Topic2  0.263858       munch  1.124604  -0.6021  -3.5757
10     Topic2  0.296730      kitten  1.698658  -0.8971  -3.4583
9      Topic2  0.271232        cute  1.701684  -0.9887  -3.5481, token_table=      Topic      Freq        Term
term                             
11        1  0.891782       adopt
4         2  0.939349         ate
0         2  0.631961      b

## Step 4: Update model with new data!

In [12]:
doc_6 = 'That cat is so cute! It looks good enough to eat.'

In [13]:
new_doc = text_process(doc_6)

In [14]:
ldamodel.update([dictionary.doc2bow(new_doc)])

In [15]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.050761  0.0       1        1  59.344154
1     -0.050761  0.0       2        1  40.655842, topic_info=     Category      Freq        Term     Total  loglift  logprob
term                                                           
9     Default  2.000000        cute  2.000000  18.0000  18.0000
15    Default  2.000000        look  2.000000  17.0000  17.0000
0     Default  0.000000      banana  0.000000  16.0000  16.0000
2     Default  3.000000         eat  3.000000  15.0000  15.0000
4     Default  0.000000         ate  0.000000  14.0000  14.0000
5     Default  0.000000   breakfast  0.000000  13.0000  13.0000
7     Default  0.000000     spinach  0.000000  12.0000  12.0000
6     Default  0.000000     smoothi  0.000000  11.0000  11.0000
3     Default  0.000000        like  0.000000  10.0000  10.0000
1     Default  0.000000    broccoli  0.000000   9.0000   9.0000
12    Default  0.000000      sister  0.000000   8.0000   8.0000
13    Default  0.000000   yesterday  0.000000   7.0000   7.0000
11    Default  0.000000       adopt  0.000000   6.0000   6.0000
8     Default  0.000000  chinchilla  0.000000   5.0000   5.0000
14    Default  0.000000     hamster  0.000000   4.0000   4.0000
17    Default  0.000000        piec  0.000000   3.0000   3.0000
16    Default  0.000000       munch  0.000000   2.0000   2.0000
10    Default  0.000000      kitten  0.000000   1.0000   1.0000
0      Topic1  0.808618      banana  0.992147   0.3173  -2.7817
4      Topic1  0.678041         ate  0.860737   0.2832  -2.9578
5      Topic1  0.677686   breakfast  0.860500   0.2830  -2.9583
7      Topic1  0.677652     spinach  0.860477   0.2830  -2.9583
6      Topic1  0.677341     smoothi  0.860270   0.2827  -2.9588
3      Topic1  0.676766        like  0.859887   0.2823  -2.9597
1      Topic1  0.677557    broccoli  0.904812   0.2326  -2.9585
12     Topic1  0.556162      sister  0.779520   0.1842  -3.1559
13     Topic1  0.555438   yesterday  0.779038   0.1835  -3.1572
11     Topic1  0.554768       adopt  0.778591   0.1829  -3.1584
8      Topic1  0.549653  chinchilla  0.775183   0.1780  -3.1677
14     Topic1  0.547954     hamster  0.774051   0.1764  -3.1708
17     Topic1  0.547757        piec  0.773920   0.1762  -3.1711
16     Topic1  0.547704       munch  0.773884   0.1761  -3.1712
10     Topic1  0.556170      kitten  0.823924   0.1288  -3.1559
2      Topic1  2.471556         eat  3.976571   0.0462  -1.6644
15     Topic1  0.665552        look  2.773111  -0.9053  -2.9764
9      Topic1  0.629340        cute  2.793379  -0.9685  -3.0323
9      Topic2  2.164039        cute  2.793379   0.6448  -1.4190
15     Topic2  2.107559        look  2.773111   0.6256  -1.4455
2      Topic2  1.505015         eat  3.976571  -0.0716  -1.7822
10     Topic2  0.267754      kitten  0.823924  -0.2240  -3.5087
16     Topic2  0.226180       munch  0.773884  -0.3301  -3.6774
17     Topic2  0.226162        piec  0.773920  -0.3302  -3.6775
14     Topic2  0.226097     hamster  0.774051  -0.3306  -3.6778
8      Topic2  0.225530  chinchilla  0.775183  -0.3346  -3.6803
11     Topic2  0.223823       adopt  0.778591  -0.3466  -3.6879
13     Topic2  0.223600   yesterday  0.779038  -0.3482  -3.6889
12     Topic2  0.223358      sister  0.779520  -0.3499  -3.6900
1      Topic2  0.227255    broccoli  0.904812  -0.4816  -3.6727
3      Topic2  0.183121        like  0.859887  -0.6466  -3.8886
6      Topic2  0.182929     smoothi  0.860270  -0.6481  -3.8897
7      Topic2  0.182825     spinach  0.860477  -0.6489  -3.8902
5      Topic2  0.182814   breakfast  0.860500  -0.6490  -3.8903
4      Topic2  0.182695         ate  0.860737  -0.6499  -3.8910
0      Topic2  0.183529      banana  0.992147  -0.7875  -3.8864, token_table=      Topic      Freq        Term
term                             
11        1  1.284371       adopt
4         1  1.161796         ate
0         1  1.007915      b

# Model 2: Yelp Reviews

In [16]:
review = pd.read_json("../yelp_academic_dataset_review.json")

In [17]:
review.head()

,business_id,date,review_id,stars,text,type,user_id,votes
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,"{'funny': 0, 'useful': 5, 'cool': 2}"
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,"{'funny': 0, 'useful': 0, 'cool': 0}"
10,AsSCv0q_BWqIe3mX2JqsOQ,2010-06-16,E11jzpKz9Kw5K7fuARWfRw,5,The oldish man who owns the store is as sweet ...,review,-OMlS6yWkYjVldNhC31wYg,"{'funny': 1, 'useful': 3, 'cool': 1}"
100,CrBsdxqOjPdnfsDxV89GJQ,2010-02-17,SmUMyCUNrT9HEo_DXdgUuQ,4,I have to admit that I find myself thinking th...,review,bZFRqP7s0Vszxeu8_IwYow,"{'funny': 0, 'useful': 1, 'cool': 0}"
1000,vfLog2bLJGl6hAFtGtr0GQ,2012-10-30,bmsk5foqmcBgQ_pylNDpCw,4,Great atmosphere with interesting lights (look...,review,3ltazFFclBfchSYlctX6iA,"{'funny': 0, 'useful': 0, 'cool': 0}"


In [18]:
review.shape

(229907, 8)

In [19]:
texts = []

In [20]:
import time

In [21]:
t0 = time.time()

for i in range(10000):
    if i % 1000 == 0:
        print("Iteration {}".format(i))
        print(str(time.time() - t0) + " seconds elapsed.")
    texts.append(text_process(review.loc[i, 'text']))

Iteration 0
0.000324249267578125 seconds elapsed.
Iteration 1000
25.63820219039917 seconds elapsed.
Iteration 2000
52.177359104156494 seconds elapsed.
Iteration 3000
79.06812024116516 seconds elapsed.
Iteration 4000
105.75904703140259 seconds elapsed.
Iteration 5000
131.3081660270691 seconds elapsed.
Iteration 6000
159.84377312660217 seconds elapsed.
Iteration 7000
188.4672031402588 seconds elapsed.
Iteration 8000
216.40954208374023 seconds elapsed.
Iteration 9000
245.6086630821228 seconds elapsed.


In [22]:
texts[0]

['wife',
 'took',
 'birthday',
 'breakfast',
 'excel',
 'weather',
 'perfect',
 'made',
 'sit',
 'outsid',
 'overlook',
 'ground',
 'absolut',
 'pleasur',
 'waitress',
 'excel',
 'food',
 'arriv',
 'quickli',
 'semi',
 'busi',
 'saturday',
 'morn',
 'look',
 'like',
 'place',
 'fill',
 'pretti',
 'quickli',
 'earlier',
 'get',
 'better',
 'favor',
 'get',
 'bloodi',
 'mari',
 'phenomen',
 'simpli',
 'best',
 'ever',
 'pretti',
 'sure',
 'use',
 'ingredi',
 'garden',
 'blend',
 'fresh',
 'order',
 'amaz',
 'everyth',
 'menu',
 'look',
 'excel',
 'white',
 'truffl',
 'scrambl',
 'egg',
 'veget',
 'skillet',
 'tasti',
 'delici',
 'came',
 '2',
 'piec',
 'griddl',
 'bread',
 'amaz',
 'absolut',
 'made',
 'meal',
 'complet',
 'best',
 'toast',
 'ever',
 'anyway',
 'wait',
 'go',
 'back']

In [26]:
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = models.ldamodel.LdaModel(corpus,                     # pass in our corpus
                                    id2word = dictionary,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 10,            # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01) # only include topics above this probability threshold

In [27]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.105603 -0.074347       1        1  23.686241
9     -0.092920 -0.061153       2        1  15.620202
2     -0.003669  0.059492       3        1  10.977431
7     -0.036074  0.021828       4        1  10.958612
8     -0.082124  0.086586       5        1  10.778649
5     -0.032859  0.073321       6        1   9.755377
3     -0.032610  0.034631       7        1   9.215709
0     -0.084551 -0.059213       8        1   5.691893
1      0.268489  0.142707       9        1   1.732571
6      0.201922 -0.223850      10        1   1.583316, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
24    Default  6266.000000       food  6266.000000  30.0000  30.0000
97    Default  4936.000000      great  4936.000000  29.0000  29.0000
811   Default  1131.000000     burger  1131.000000  28.0000  28.0000
341   Default  1702.000000        bar  1702.000000  27.0000  27.0000
1344  Default   664.000000      sushi   664.000000  26.0000  26.0000
292   Default  2207.000000    restaur  2207.000000  25.0000  25.0000
110   Default  1435.000000      pizza  1435.000000  24.0000  24.0000
483   Default   989.000000      store   989.000000  23.0000  23.0000
451   Default  1404.000000       hour  1404.000000  22.0000  22.0000
135   Default  1859.000000         us  1859.000000  21.0000  21.0000
45    Default  7343.000000      place  7343.000000  20.0000  20.0000
450   Default  1094.000000      happi  1094.000000  19.0000  19.0000
95    Default  6806.000000       good  6806.000000  18.0000  18.0000
1125  Default  1512.000000        fri  1512.000000  17.0000  17.0000
913   Default   984.000000       beer   984.000000  16.0000  16.0000
329   Default   906.000000       wine   906.000000  15.0000  15.0000
567   Default  1593.000000      night  1593.000000  14.0000  14.0000
499   Default   756.000000       roll   756.000000  13.0000  13.0000
114   Default  2061.000000      price  2061.000000  12.0000  12.0000
505   Default  1725.000000    chicken  1725.000000  11.0000  11.0000
299   Default  3162.000000     servic  3162.000000  10.0000  10.0000
294   Default  1465.000000      salad  1465.000000   9.0000   9.0000
39    Default  3614.000000      order  3614.000000   8.0000   8.0000
474   Default   616.000000        ice   616.000000   7.0000   7.0000
419   Default   728.000000       taco   728.000000   6.0000   6.0000
222   Default  1400.000000        ask  1400.000000   5.0000   5.0000
118   Default  1369.000000       sauc  1369.000000   4.0000   4.0000
85    Default  1807.000000      drink  1807.000000   3.0000   3.0000
1058  Default   846.000000       room   846.000000   2.0000   2.0000
37    Default  1747.000000       menu  1747.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
4478  Topic10    10.256532      satay    11.129458   4.0640  -6.9354
5016  Topic10    10.121477       user    10.984490   4.0638  -6.9486
812   Topic10    46.823151      china    53.775242   4.0072  -5.4169
876   Topic10    78.623909        dim    95.213409   3.9542  -4.8986
3757  Topic10    17.248716     airway    19.282238   4.0342  -6.4155
501   Topic10    86.405899  vietnames   108.989883   3.9134  -4.8042
2825  Topic10    80.559395        sum   107.407112   3.8580  -4.8743
1793  Topic10   176.022003     chines   274.971191   3.6996  -4.0927
1362  Topic10   105.275185      asian   178.133179   3.6197  -4.6067
1719  Topic10    69.376564        san   120.830002   3.5908  -5.0237
432   Topic10    49.998234     flight   110.381569   3.3537  -5.3513
357   Topic10    19.663685      diego    27.440298   3.8124  -6.2845
722   Topic10    19.160521     airlin    27.242023   3.7937  -6.3104
5506  Topic10    40.582588     cuisin   115.102394   3.1032  -5.5599
870   Topic10    60.438507    authent   247.867004   2.7344  -5.16

In [25]:
for i in ldamodel.print_topics(num_topics=40, num_words=5):
    print(i)

(0, '0.039*"ice" + 0.037*"cream" + 0.031*"chocol" + 0.023*"flavor" + 0.022*"cake"')
(1, '0.036*"class" + 0.017*"school" + 0.013*"year" + 0.011*"festiv" + 0.010*"student"')
(2, '0.014*"nail" + 0.009*"time" + 0.008*"use" + 0.008*"well" + 0.008*"call"')
(3, '0.022*"yogurt" + 0.022*"gyro" + 0.018*"frozen" + 0.016*"hike" + 0.015*"pet"')
(4, '0.104*"dog" + 0.020*"hot" + 0.016*"ny" + 0.015*"park" + 0.012*"can"')
(5, '0.031*"pho" + 0.030*"noodl" + 0.021*"roll" + 0.016*"spring" + 0.016*"asian"')
(6, '0.023*"buffet" + 0.017*"food" + 0.015*"indian" + 0.012*"place" + 0.012*"restaur"')
(7, '0.027*"fri" + 0.022*"burger" + 0.016*"good" + 0.015*"place" + 0.015*"like"')
(8, '0.035*"game" + 0.034*"mall" + 0.016*"place" + 0.015*"park" + 0.015*"watch"')
(9, '0.012*"spa" + 0.011*"amaz" + 0.011*"beauti" + 0.011*"resort" + 0.010*"design"')
(10, '0.072*"breakfast" + 0.036*"egg" + 0.019*"toast" + 0.017*"brunch" + 0.016*"bacon"')
(11, '0.023*"us" + 0.021*"server" + 0.020*"hour" + 0.016*"happi" + 0.013*"food"')
